In [1]:
# Notebook to compare basic meteorological parameters between WRF simulations and ERA5 data
# Some of the following imports are not used right now, but will retain for future flexibility
import os
import subprocess
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.cbook as cbook
from matplotlib.colors import Normalize
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.colors as mcolors
import pandas as pd
import xarray as xr
import numpy as np
import math
from numpy import *
from pylab import *
import pygrib
import pyproj

from siphon.catalog import TDSCatalog
from siphon.http_util import session_manager
from datetime import datetime, timedelta
from xarray.backends import NetCDF4DataStore
from netCDF4 import Dataset
import metpy as metpy
import metpy.calc as mpcalc
from metpy.plots import ctables
from metpy.units import units
from metpy.plots import add_metpy_logo, add_timestamp
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from cartopy import config
import wrf
from wrf import (to_np, interplevel, geo_bounds, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                          facecolor="none", name="admin_1_states_provinces_shp")
import glob

In [2]:
"""
Data user settings, constants, and level ranges
"""
################## wrfout files path ###################################################

wrf_file = "/home/jupyter-sdsmit12@ncsu.edu/WRF_project_1/wwrf/02_04_2001/wrf*"

################## Path to where ERA files reside ######################################

era = '/scratch/sawyer/ERA5/ERA5_02_04_2001/ERA5_data.grib'

################### Saving plots option ##################

save = True

########### If you want to save plots, specify the subdirectory you want them in ###########

plotsdir = "/home/jupyter-sdsmit12@ncsu.edu/WRF_project_1/wwrf/02_04_2001/plots/"

############ Gravity ###########
g0 = 9.80665

############### Select a pressure level #################
p_level = 850

######## Number of vertical levels in GRIB data #########

num_levels = 37

####### The map projection you would like to use ########

projection=ccrs.PlateCarree(central_longitude=0) 

################ Set bounds for plotting #################

wlon = -175
elon = -110
slat = 20
nlat = 60

############################# Set ranges and intervals for plotting ###############################

if p_level == 1000:
    levels = np.arange(-180,180,30)
    q_levels = np.arange(0, 22, 1)
    wnd_levels = np.arange(0, 30, 1)
elif p_level == 850:
    levels = np.arange(1000, 2000, 50)
    q_levels = np.arange(0, 16, .5)
    wnd_levels = np.arange(10, 51, 1)    
elif p_level == 700:
    levels = np.arange(2350, 3150, 30)
    q_levels = np.arange(0, 12, .5)
    wnd_levels = np.arange(10, 50, 1) 
elif p_level == 500:
    levels = np.arange(4000, 6000, 60)
    q_levels = np.arange(0, 16, .5)
    wnd_levels = np.arange(10, 65, 1) 
elif p_level == 300:
    levels = np.arange(8200, 9600, 120)
    wnd_levels = np.arange(20, 90, 1) 
elif p_level == 200:
    levels = np.arange(10800,12300, 120)
    wnd_levels = np.arange(20, 90, 1) 

ivt_levels = np.arange(250,1450,100)
pv_levels = np.arange(0,1,.1)

########### Define the colors in the desired sequence for IVT: ###########

colors = ['#FFFF00', '#FFEE00','#FFDC00', '#FFB700',
          '#FFA300', '#FF9000', '#FF7D00', '#FF6800',
          '#FF5200', '#C70039','#900C3F', (.88,.24,.69)]

# Create a colormap using ListedColormap
cmap = mcolors.ListedColormap(colors)

In [3]:
"""
This function computes Integrated Vapor Transport (IVT) using data in GRIB format
"""
def calculate_ERA_IVT():
    
    # Get the shape of the data (latitude, longitude, level)
    first = len(grib.values[:])
    second = len(grib.values[0,:])

    # Create an empty array to store the specific humidity, wind components and pressure at every level
    pressure_array = np.zeros([num_levels, first, second])
    sp_hum_array = np.zeros([num_levels, first, second])
    uwnd_array = np.zeros([num_levels, first, second])
    vwnd_array = np.zeros([num_levels, first, second])
    #print(sp_hum_array.shape)
    # Loop over all levels and extract the specific humidity
    for i, level in enumerate(grbs.select(name='Specific humidity', time = hours, date = dates)):
        # Extract the specific humidity for this level
        sp_hum = level.values
        # Store the specific humidity in the array
        pressure_array[i,:,:] = level['level']*100
        sp_hum_array[i,:,:] = sp_hum
    for i, level in enumerate(grbs.select(name='U component of wind', time = hours, date = dates)):

        u_wind = np.abs(level.values)
        uwnd_array[i,:,:] = u_wind

    for i, level in enumerate(grbs.select(name='V component of wind', time = hours, date = dates)):

        v_wind =np.abs(level.values)
        vwnd_array[i,:,:] = v_wind 
        
    uflux_l = []
    vflux_l = []

    for m in range(0,len(sp_hum_array)-2):
        layer_diff = pressure_array[m,:,:]-pressure_array[m+1,:,:]
        ql = (sp_hum_array[m+1,:,:]+sp_hum_array[m,:,:])/2
        ul = (uwnd_array[m+1,:,:]+uwnd_array[m,:,:])/2
        vl = (vwnd_array[m+1,:,:]+vwnd_array[m,:,:])/2
        qfl = (ql/9.80665)*layer_diff
        uflux= ul * qfl
        vflux = vl * qfl
        uflux_l.append(uflux)
        vflux_l.append(vflux)

    uflux_l=np.asarray(uflux_l)
    vflux_l=np.asarray(vflux_l)
    uIVT = np.sum(uflux_l, axis = 0)
    vIVT = np.sum(vflux_l, axis = 0)
    IVT_tot=np.sqrt(uIVT**2+vIVT**2)
    IVT_tot=xr.DataArray(IVT_tot,dims=['lat','lon'])
    
    return IVT_tot
    
"""
This function computes Integrated Vapor Transport (IVT) from wrfout files
"""
def calculate_wrf_IVT():
   
    ua = getvar(ncfile, "ua")
    ua = np.abs(ua)
    va = getvar(ncfile, "va")
    va = np.abs(va)
    p = getvar(ncfile, "pressure")*100
    mixing_ratio = ncfile['QVAPOR'][0,:,:,:]
    
    uflux_l = []
    vflux_l = []
    for m in range(0,len(mixing_ratio)-2):
        layer_diff = p[m,:,:]-p[m+1,:,:]
        ql = (mixing_ratio[m+1,:,:]+mixing_ratio[m,:,:])/2
        ul = (ua[m+1,:,:]+ua[m,:,:])/2
        vl = (va[m+1,:,:]+va[m,:,:])/2
        qfl = (ql/9.80665)*layer_diff
        uflux= ul * qfl
        vflux = vl * qfl
        uflux_l.append(uflux)
        vflux_l.append(vflux)

    uflux_l=np.asarray(uflux_l)
    vflux_l=np.asarray(vflux_l)
    uIVT = np.sum(uflux_l, axis = 0)
    vIVT = np.sum(vflux_l, axis = 0)
    IVT_tot=np.sqrt(uIVT**2+vIVT**2)
    IVT_tot=xr.DataArray(IVT_tot,dims=['lat','lon'])
    
    return IVT_tot

In [4]:
# Set up for saving graphics files, create directory for plots, if needed
if os.path.isdir(plotsdir) != 1:
    subprocess.call(["mkdir","-p",plotsdir])
os.chdir(plotsdir)

# Lets see whats in our GRIB file

grbs = pygrib.open(era)
grib = grbs.select(name='Specific humidity')[0]

# Get the data and latitudes/longitudes
coords = grbs.select(name='Geopotential', level=p_level)[0]
lats, lons = coords.latlons()

# Prepare files for processing

datafiles = (glob.glob(wrf_file))
datafiles.sort()
numfiles=len(datafiles)
print(numfiles)

7


In [5]:
for i in range(0,numfiles):

    ncfile = Dataset(datafiles[i])
    Time=wrf.extract_times(ncfile, timeidx=0, method='cat', squeeze=True, cache=None, meta=False, do_xtime=False)
    timestr=(str(Time))
    
    # Set up one time string for plot titles, another for file names
    titletime=(timestr[0:10]+' '+timestr[11:16])
    filetime=(timestr[0:10]+'_'+timestr[11:13])
    print('WRF valid time: ',filetime)
    plot_filetime = (timestr[0:10]+' '+timestr[11:13])
    dates = int(timestr[0:10].replace('-', ''))
    
    # Multiply by 100 to get time in correct format for grib files
    hours = int(timestr[11:13]) * 100
    
    # Get all the variables we need from wrf
    p = getvar(ncfile, "pressure")
    
    height = getvar(ncfile, "z")
    heights = interplevel(height, p, p_level)
    
    wspd = getvar(ncfile, "wspd_wdir")[0,:]
    wrf_windsp = interplevel(wspd, p, p_level)
    
    # Get the lat/lon coordinates and fix lons around dateline from wrf output
    wrf_lats, wrf_lons = latlon_coords(height)
    new_lons = np.where(wrf_lons > 0, wrf_lons - 360, wrf_lons)
    
    # Get all the variables we need from ERA
    z = grbs.select(name='Geopotential', level = p_level, time = hours, date = dates)[0].values / g0
    uwnd = grbs.select(name='U component of wind', level = p_level, time = hours, date = dates)[0].values 
    vwnd = grbs.select(name='V component of wind', level = p_level, time = hours, date = dates)[0].values
    uwind = uwnd * units.meter / units.second
    vwind = vwnd * units.meter / units.second
    
    # calculate wind speed for ERA5 data
    era_windsp =  metpy.calc.wind_speed(uwind, vwind)
    
    #calculate IVT
    era_ivt = calculate_ERA_IVT()
    wrf_ivt = calculate_wrf_IVT()

    # Set up the plot
   
    fig, axes = plt.subplots(2,2,subplot_kw={'projection': projection},figsize=(16, 14))
    
    # dark brown for state/coastlines
    dark_brown = (0.4, 0.2, 0)
    
    for x in range(0, 2):
        for y in range(0,2):
            axes[x,y].set_extent([wlon,elon,slat,nlat], crs = ccrs.PlateCarree())
            axes[x,y].add_feature(cfeature.STATES, edgecolor=dark_brown) #Add US states
            axes[x,y].add_feature(cfeature.COASTLINE, edgecolor=dark_brown)#Add coastlines
            
            gl = axes[x,y].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
            gl.top_labels=False   # suppress top labels
            gl.right_labels=False 
    
        # Plot the contours
        cs = axes[x,0].contour(lons, lats, z, levels=levels, colors='black')
        cs2 = axes[x,1].contour(new_lons, wrf_lats, heights, levels=levels, colors='black')
        axes[x,0].clabel(cs, fmt='%1.0f')
        axes[x,1].clabel(cs2, fmt='%1.0f', inline_spacing=5)

    cs_sp = axes[0,0].contourf(lons, lats, era_windsp,levels=wnd_levels, vmax=50, cmap='jet')
    cs_sp2 = axes[0,1].contourf(new_lons, wrf_lats, wrf_windsp,levels=wnd_levels, vmax=50, cmap='jet')
    #Add a colorbar

    axes[0,0].set_title('ERA5 ' + str(p_level) +' mb Geopotential Heights and Wind Speed \n ' + str(plot_filetime)+ ' UTC',fontsize=16)
    axes[0,1].set_title('WWRF Reforecast ' + str(p_level)+' mb Geopotential Heights and Wind Speed \n '+ str(plot_filetime)+ ' UTC', fontsize=16)

    cs_ivt = axes[1,0].contourf(lons, lats, era_ivt,levels=ivt_levels, vmax=1450, cmap=cmap)
    cs_ivt2 = axes[1,1].contourf(new_lons, wrf_lats, wrf_ivt,levels=ivt_levels, vmax=1450, cmap=cmap)

    # Set the plot title
    axes[1,0].set_title('ERA5 ' + str(p_level) +' mb Geopotential Heights and IVT \n '+ str(plot_filetime)+ ' UTC', fontsize=16)
    axes[1,1].set_title('WWRF Reforecast ' + str(p_level)+' mb Geopotential Heights and IVT \n'+ str(plot_filetime)+ ' UTC',fontsize=16)
            
    wind_cbar = plt.colorbar(cs_sp, orientation = 'horizontal', shrink = 0.8)
    wind_cbar2 = plt.colorbar(cs_sp2, orientation = 'horizontal', shrink = 0.8)
    wind_cbar.set_label("wind speed (m$s^{-1}$)", fontsize = 12)
   
    wind_cbar2.set_label("wind speed (m$s^{-1}$)", fontsize = 12)
    #Add a colorbar

    ivt_cbar = plt.colorbar(cs_ivt, orientation = 'horizontal', shrink = 0.8)
    ivt_cbar2 = plt.colorbar(cs_ivt2, orientation = 'horizontal', shrink = 0.8)
    ivt_cbar.set_label("IVT (kg $m^{-1}$$s^{-1}$)", fontsize = 12)
    ivt_cbar2.set_label("IVT (kg $m^{-1}$$s^{-1}$)", fontsize = 12)
    
    if save == True:
        # Create separate plot file and save as .png, then show and close
        outTPlotName= str(plot_filetime)+'.png'
        plt.savefig(plotsdir+'/'+outTPlotName)

        plt.close(fig)
    else:
        plt.show()

WRF valid time:  2001-01-29_00
WRF valid time:  2001-01-30_00
WRF valid time:  2001-01-31_00
WRF valid time:  2001-02-01_00
WRF valid time:  2001-02-02_00
WRF valid time:  2001-02-03_00
WRF valid time:  2001-02-04_00
